In [2]:
import pandas as pd
df = pd.read_csv("data/wdi_gdp_data.csv")
df.drop(["Unnamed: 0", "youth_literacy_gpi"],axis=1, inplace=True)
df = df[df["year"] != 2024]
df.to_csv("data/temp.csv")
print(df.shape)


(10176, 14)


In [3]:
def fill_internet_users_before_first(df):
    df = df.copy()
    
    for country, g in df.groupby("name"):
        first_valid = g["internet_users_pct"].first_valid_index()
        if first_valid is not None:
            # Get all indices for this country that are before first_valid
            mask = g.index[g.index < first_valid]
            # Set those to 0 if they are NaN
            df.loc[mask, "internet_users_pct"] = df.loc[mask, "internet_users_pct"].fillna(0)
    return df
df = fill_internet_users_before_first(df)

In [ ]:
x = 0.1  # for example, drop countries with >10% missing values overall
missing_frac = df.groupby("name").apply(lambda g: g.isna().mean().mean())

remove_countries = missing_frac[missing_frac >= x].index

print(df.shape)
df = df[~df["name"].isin(remove_countries)]
print(df.shape)

0
(8403, 14)
(8403, 14)


C:\Users\tvuontis\AppData\Local\Temp\ipykernel_19632\1830951871.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  missing_frac = df.groupby("name").apply(lambda g: g.isna().mean().mean())


In [9]:
# Count missing HCI per country
missing_hci = df[df["human_capital_index"].isna()]
missing_count = missing_hci["name"].value_counts()

# Count total rows per country
total_count = df["name"].value_counts()

# Combine into one DataFrame
country_missing = (
    pd.DataFrame({
        "missing_hci": missing_count,
        "total_rows": total_count
    })
    .fillna(0)
    .astype(int)
    .sort_values("missing_hci", ascending=False)
)

# Add share of missing
country_missing["missing_share_%"] = (
    100 * country_missing["missing_hci"] / country_missing["total_rows"]
).round(1)

missing_hci_countries  = list(country_missing[country_missing["missing_share_%"] > 0].head(20).index)
missing_hci_countries += ['Bosnia and Herzegovina', 'Haiti']
df = df[~df["name"].isin(missing_hci_countries)]
df.shape

(8403, 14)

In [12]:
def drop_leading_nans(group, col="infant_mortality_rate"):
    # Find the first index where value is not NaN
    first_valid = group[col].first_valid_index()
    # If the whole column is NaN, drop the group entirely
    if first_valid is None:
        return pd.DataFrame(columns=group.columns)
    # Otherwise keep rows from the first valid value onward
    return group.loc[first_valid:]
    
df = (
    df.sort_values(["name", "year"])
      .groupby("name", group_keys=False)
      .apply(drop_leading_nans)
      .reset_index(drop=True)
)
df.shape

C:\Users\tvuontis\AppData\Local\Temp\ipykernel_19632\2123377202.py:13: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(drop_leading_nans)


(8403, 14)

In [7]:
def drop_trailing_nans(group, col="internet_users_pct"):
    # Find the last index where value is not NaN
    last_valid = group[col].last_valid_index()
    # If everything is NaN, drop the group entirely
    if last_valid is None:
        return pd.DataFrame(columns=group.columns)
    # Keep rows from start up to the last valid value
    return group.loc[:last_valid]

df = (
    df.sort_values(["name", "year"])
      .groupby("name", group_keys=False)
      .apply(drop_trailing_nans)
      .reset_index(drop=True)
)
df.to_csv("data/clean_data.csv")
total_nans = df.isna().sum().sum()
print("Total NaN values:", total_nans)

df.shape

Total NaN values: 0


C:\Users\tvuontis\AppData\Local\Temp\ipykernel_19632\3977114810.py:13: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(drop_trailing_nans)


(8403, 14)